In [1]:
!nvidia-smi

Wed Jan 19 16:00:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! gdown --id 1i2x7osfloYkXDHMRn0WuVs2P3m751cj1
! pip -q uninstall -y kaggle
! pip -q install --upgrade pip
! pip -q install kaggle --upgrade
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download jigsaw-toxic-severity-rating
! kaggle datasets download julian3833/jigsaw-toxic-comment-classification-challenge
! kaggle datasets download rajkumarl/ruddit-jigsaw-dataset

Downloading...
From: https://drive.google.com/uc?id=1i2x7osfloYkXDHMRn0WuVs2P3m751cj1
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 135kB/s]
     |████████████████████████████████| 1.7 MB 14.1 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s             
  Preparing metadata (setup.py) ... done
  0% 0.00/6.72M [00:00<?, ?B/s]
100% 6.72M/6.72M [00:00<00:00, 105MB/s]
 90% 48.0M/53.4M [00:00<00:00, 264MB/s]
100% 53.4M/53.4M [00:00<00:00, 260MB/s]
  0% 0.00/7.42M [00:00<?, ?B/s]
100% 7.42M/7.42M [00:00<00:00, 154MB/s]


In [3]:
!mkdir JTCC
!unzip /content/jigsaw-toxic-comment-classification-challenge.zip -d JTCC
!rm /content/jigsaw-toxic-comment-classification-challenge.zip
!mkdir RJD
!unzip /content/ruddit-jigsaw-dataset.zip -d RJD
!rm /content/ruddit-jigsaw-dataset.zip
!mkdir Data
!unzip /content/jigsaw-toxic-severity-rating.zip -d Data
!rm /content/jigsaw-toxic-severity-rating.zip

Archive:  /content/jigsaw-toxic-comment-classification-challenge.zip
  inflating: JTCC/sample_submission.csv  
  inflating: JTCC/test.csv           
  inflating: JTCC/test_labels.csv    
  inflating: JTCC/train.csv          
Archive:  /content/ruddit-jigsaw-dataset.zip
  inflating: RJD/Dataset/ReadMe.md   
  inflating: RJD/Dataset/Ruddit.csv  
  inflating: RJD/Dataset/Ruddit_individual_annotations.csv  
  inflating: RJD/Dataset/Thread_structure.txt  
  inflating: RJD/Dataset/create_dataset_variants.py  
  inflating: RJD/Dataset/identityterms_group.txt  
  inflating: RJD/Dataset/load_node_dictionary.py  
  inflating: RJD/Dataset/node_dictionary.npy  
  inflating: RJD/Dataset/post_with_issues.csv  
  inflating: RJD/Dataset/ruddit_with_text.csv  
  inflating: RJD/Dataset/sample_input_file.csv  
  inflating: RJD/LICENSE             
  inflating: RJD/Models/BERT.py      
  inflating: RJD/Models/BiLSTM.py    
  inflating: RJD/Models/HateBERT.py  
  inflating: RJD/Models/README.md    
  infla

In [4]:
! pip -q install sentencepiece
! pip -q install transformers
! pip -q install wandb --upgrade
! pip -q install colorama

     |████████████████████████████████| 1.2 MB 15.2 MB/s            
     |████████████████████████████████| 3.4 MB 14.3 MB/s            
     |████████████████████████████████| 895 kB 58.5 MB/s            
     |████████████████████████████████| 3.3 MB 51.0 MB/s            
     |████████████████████████████████| 67 kB 5.9 MB/s             
     |████████████████████████████████| 596 kB 64.2 MB/s            
     |████████████████████████████████| 1.7 MB 15.6 MB/s            
     |████████████████████████████████| 180 kB 60.7 MB/s            
     |████████████████████████████████| 97 kB 7.4 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 142 kB 62.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 63 kB 2.1 MB/s             


In [5]:
import os
import gc
import copy
import time
import random
import string
import wandb

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
pd.read_csv("RJD/Dataset/ruddit_with_text.csv").head()

,post_id,comment_id,txt,url,offensiveness_score
0,42g75o,cza1q49,> The difference in average earnings between m...,https://www.reddit.com/r/changemyview/comments...,-0.083
1,42g75o,cza1wdh,"The myth is that the ""gap"" is entirely based o...",https://www.reddit.com/r/changemyview/comments...,-0.022
2,42g75o,cza23qx,[deleted],https://www.reddit.com/r/changemyview/comments...,0.167
3,42g75o,cza2bw8,The assertion is that women get paid less for ...,https://www.reddit.com/r/changemyview/comments...,-0.146
4,42g75o,cza2iji,You said in the OP that's not what they're mea...,https://www.reddit.com/r/changemyview/comments...,-0.083


In [7]:
Config = {"seed": 2021,
          "csv_path":"RJD/Dataset/ruddit_with_text.csv",
          "epochs": 5,
          "model_name": "roberta-base",
          "train_batch_size": 64,
          "valid_batch_size": 512,
          "num_workers": 4,
          "max_length": 128,
          "lr": 1e-5,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_0":20,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          'wandb': True,
          'apex': False,
          'n_accum': 1,
          "debug":False
          }

Config["tokenizer"] = AutoTokenizer.from_pretrained(Config['model_name'])

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [8]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(Config['seed'])

In [9]:
# Start W&B logging
if Config['wandb']:
    run = wandb.init(
        project='pytorch',
        config=Config,
        group='nlp',
        job_type='train',
        anonymous='must'
    )

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
# Some utility functions
def wandb_log(**kwargs):
    """
    Logs a key-value pair to W&B
    """
    for k, v in kwargs.items():
        wandb.log({k: v})

In [18]:
def create_folds(data, num_splits = Config["n_fold"]):
    data["kfold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data))))
    data.loc[:, "bins"] = pd.cut(
        data["offensiveness_score"], bins=num_bins, labels=False
    )
    kf = StratifiedKFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    data = data.drop("bins", axis=1)
    return data

In [12]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer=Config["tokenizer"], max_length=Config["max_length"]):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.comment_text = df['txt'].values
        self.score = df['offensiveness_score'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        comment_text = self.comment_text[index]
        score = self.score[index]
        inputs = self.tokenizer.encode_plus(
                                comment_text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
        input_ids = inputs['input_ids']
        input_mask = inputs['attention_mask']
        
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'input_mask': torch.tensor(input_mask, dtype=torch.long),
            'score': torch.tensor(score, dtype=torch.float)
        }

In [13]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768,Config['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs

In [14]:
def criterion(outputs, targets):
    return nn.MSELoss()(outputs, targets)

In [15]:
class Trainer:
    def __init__(self, config, dataloaders, optimizer, model, loss_fns, scheduler, device=Config['device'], apex=Config['apex']):
        self.train_loader, self.valid_loader = dataloaders
        self.train_loss_fn, self.valid_loss_fn = loss_fns
        self.scheduler = scheduler
        self.optimizer = optimizer
        self.model = model
        self.device = device
        self.apex = apex
        self.Config = config
    
    def train_one_epoch(self):
        if self.apex:
            scaler = GradScaler()

        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        train_preds, train_labels = [], []
        dataset_size = 0
        running_loss = 0.0

        for step, data in train_pbar:        
            input_ids = data['input_ids'].to(self.device, dtype = torch.long)
            input_mask = data['input_mask'].to(self.device, dtype = torch.long)
            scores = data['score'].to(self.device, dtype=torch.float)
            
            batch_size = input_ids.size(0)

            if self.apex:
                with(autocast(enabled=True)):

                    outputs = model(input_ids, input_mask)
                    
                    loss = criterion(outputs, scores)
                    loss = loss / self.Config["n_accum"]
                
                scaler.scale(loss).backward()

                if (step + 1) % self.Config["n_accum"] == 0:
                    scaler.step(self.optimizer)
                    scaler.update()
                    optimizer.zero_grad()

                    if self.scheduler:
                        self.scheduler.step()

                running_loss += (loss.item() * batch_size)
                dataset_size += batch_size
            
            else:
                outputs = model(input_ids, input_mask)
                    
                loss = criterion(outputs, scores)
                loss = loss / self.Config["n_accum"]

                loss.backward()
                loss = loss / self.Config["n_accum"]
                if (step + 1)%self.Config["n_accum"] == 0:
                    optimizer.step()
                    optimizer.zero_grad()

                    if self.scheduler:
                        self.scheduler.step()

                running_loss += (loss.item() * batch_size)
                dataset_size += batch_size

            epoch_loss = running_loss / dataset_size                
            
            train_pbar.set_postfix(Train_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])   
        
        gc.collect()
        
        return epoch_loss

    @torch.no_grad()
    def valid_one_epoch(self):

        self.model.eval()
        valid_pbar = tqdm(enumerate(self.valid_loader), total=len(self.valid_loader))
        dataset_size = 0
        running_loss = 0.0

        for step, data in valid_pbar:
            input_ids = data['input_ids'].to(self.device, dtype = torch.long)
            input_mask = data['input_mask'].to(self.device, dtype = torch.long)
            scores = data['score'].to(self.device, dtype=torch.float)
            
            batch_size = input_ids.size(0)

            outputs = model(input_ids, input_mask)
                    
            loss = criterion(outputs, scores)
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size
            epoch_loss = running_loss / dataset_size
            
            valid_pbar.set_postfix(Valid_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])
        
            gc.collect()

        return epoch_loss
            

    def fit(self, fold: str, epochs: int = 10, output_dir: str = "/content/models/", custom_name: str = 'model.pth'):
        """
        Low-effort alternative for doing the complete training and validation process
        """
        best_loss = int(1e+7)
        for epx in range(epochs):
            print(f"{'='*20} Epoch: {epx+1} / {epochs} {'='*20}")

            train_loss = self.train_one_epoch()
            print(f"Training loss: {train_loss:.4f}")

            valid_loss = self.valid_one_epoch()
            print(f"Validation loss: {valid_loss:.4f}")

            if valid_loss < best_loss:
                best_loss = valid_loss
                self.save_model(output_dir, custom_name)
                print(f"Saved model with val_loss: {best_loss:.4f}")
            

            # Log
            if Config['wandb']:
                wandb_log(
                    train_loss=train_loss,
                    val_loss=valid_loss
                )

    def save_model(self, path, name, verbose=False):
        """
        Saves the model at the provided destination
        """
        try:
            if not os.path.exists(path):
                os.makedirs(path)
        except:
            print("Errors encountered while making the output directory")

        torch.save(self.model.state_dict(), os.path.join(path, name))
        if verbose:
            print(f"Model Saved at: {os.path.join(path, name)}")

In [ ]:
if __name__ == '__main__':
    train_file = pd.read_csv(Config['csv_path'])
    train_file = create_folds(train_file)
    
    if Config['debug']:
        train_file=train_file[:200]
        
    for fold_ in range(Config["n_fold"]):
        print(f"{'='*40} Fold: {fold_+1} / {Config['n_fold']} {'='*40}")
        
        train_ = train_file.loc[train_file["kfold"]!=fold_]
        valid_ = train_file.loc[train_file["kfold"]==fold_]
        
        train_set = JigsawDataset(
            df = train_
        )
        valid_set = JigsawDataset(
            df = valid_
        )
        
        train_loader = DataLoader(
            train_set,
            batch_size = Config['train_batch_size'],
            shuffle = True,
            num_workers = Config['num_workers'],
            pin_memory = True
        )
        
        valid_loader = DataLoader(
            valid_set,
            batch_size = Config['valid_batch_size'],
            shuffle = False,
            num_workers = Config['num_workers'],
        )
        
        model = JigsawModel(model_name=Config['model_name'])
        model = model.to(torch.device(Config['device']))
        if Config['wandb']:
            wandb.watch(model)
            
        optimizer = torch.optim.AdamW(model.parameters(), lr=Config['lr'])
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, 
            T_0=Config['T_0'], 
            eta_min=Config['min_lr']
        )
        train_lfn, valid_lfn = criterion, criterion
        
        trainer = Trainer(
            config = Config,
            dataloaders=(train_loader, valid_loader),
            loss_fns=(train_lfn, valid_lfn),
            optimizer=optimizer,
            model = model,
            scheduler=scheduler,
            apex=True
        )
        
        trainer.fit(
            fold = fold_,
            epochs = Config['epochs'],
            custom_name = f"{Config['model_name']}_{fold_}_model.bin"
        )

======================================== Fold: 1 / 5 ========================================


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


==================== Epoch: 1 / 5 ====================


100%|██████████| 73/73 [00:18<00:00,  3.86it/s, LR=3.46e-6, Train_Loss=0.117]


...

100%|██████████| 73/73 [00:18<00:00,  3.90it/s, LR=4.11e-6, Train_Loss=0.115]


Training loss: 0.1151


100%|██████████| 3/3 [00:03<00:00,  1.11s/it, LR=4.11e-6, Valid_Loss=0.11]

Validation loss: 0.1100
==================== Epoch: 5 / 5 ====================



 93%|█████████▎| 68/73 [00:17<00:01,  4.04it/s, LR=1e-5, Train_Loss=0.115]

In [ ]:
!zip -r /content/drive/MyDrive/JS_roberta-large.zip models

In [ ]:
# !rm -r /content/models